## Equações de Bellman dos jogadores para o modelo de Dou et al (2019)

In [7]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [8]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import numba as nb
from numba import jit, njit, jitclass, prange, float64, int32
%matplotlib inline


### Parâmetros do jogo

In [9]:
μ = 4.566 #número de meses entre períodos, não entendi onde entra ainda
# ρ = 0.884 # (1 - ρ) é a taxa de depreciação da empresa a cada período
ρ = 0.5 #testando nova taxa de depreciação
β = 9.84 #usamos aqui a distribuição Uniforme, depois vamos mudar isto
c0 = 0.044 #custo fixo de ir para a corte
c1 = 0.015 #custo variável de ir para a corte


hs0 = 0.28 #habilidade inicial de s
hj0 = 0.36 #habilidade inicial de j

λj = 0.346



Valores que virão dos dados

In [10]:
Vmax = float64(1.0) #valor máximo de reorganização da firma
L = float64(0.25) #valor inicial de liquidação da firma
Ds = float64(0.28) #valor da dívida com credor sênior
Dj = float64(0.35) #valor da dívida com credor júnior

D = Ds + Dj #valor total da dívida, usada para escalar custos, por exemplo


Definições iniciais: custo, valores de liquidação, valor máximo de reorganização da firma a cada período, número de rounds

In [11]:
#number of periods
def max_turns(Vmax, L, ρ):
    
    T = (np.log(L) - np.log(Vmax))/np.log(ρ) + 1
    
    return int32(T)

T = max_turns(Vmax, L, ρ)
T

3

In [12]:
#função custo
def Ct(t):
    if(t == 0):
        return 0
    else:
        return c0 * D + c1 * t * D
    
#test
Ct(0)    
Ct(1)


#array com custos em cada período
C = np.empty(T)

for t in range(T):
    C[t] = Ct(t)

0

0.037169999999999995

In [13]:
#value of the firm in each period

def Vt(Vmax, Tmax, ρ):
    
    V = np.empty(Tmax)
    
    V[0] = Vmax
    
    for t in range(1, Tmax):
        V[t] = Vmax * ρ**(t-1) - Ct(t)
        
    return V

V = Vt(Vmax, T, ρ)
V
    
        
        
    

array([1.     , 0.96283, 0.45338])

In [14]:
#valores de liquidação

def s_L(t):
    
    return np.minimum(L - Ct(t), Ds)

s_L(1)

def j_L(t):
    
    return np.minimum(L - Ct(t) - s_L(t), Dj)
    
j_L(0)

0.21283000000000002

0.0

Habilidades possíveis dos jogadores e funções de massa de probabilidade

In [15]:
#grid terá 100 espaços porque queremos duas casas decimais
grid = 100

hlow = 0.01
hhigh = 1.0

#começa no menor valor possível, vai até o maior valor possível num intervalo do tamanho do grid
hvals = np.linspace(hlow, hhigh, grid)

hvals
hvals[6]


array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

0.06999999999999999

In [16]:
def cdf(x, lt):
    
    #return the cdf of x given the lower bound lt
    #geq than 1 because of our discretization method 
    if(x >=1):
        
        return 1
    
    else:
        
        if(x >= lt):
            return 1 - ((1-x)**β)/((1-lt)**β)
        else:
            return 0
        
#test      
cdf(0.5, 1)
cdf(0.5, 0.6)
            
            

0

0

In [26]:
def pmf_cdf(lt):
    
    pmf = np.empty(grid)
    
    for i, h in enumerate(hvals):
        if(i == 0):
            pmf[i] = cdf(hvals[i], lt)
        else:
            pmf[i] = cdf(hvals[i], lt) - cdf(hvals[i-1], lt)
        
    return pmf


#test
np.sum(pmf_cdf(0.01))

pmf_cdf(0.01)
        
np.sum(np.round(pmf_cdf(0.01), 19))
    
    

1.0

array([0.00000000e+00, 9.50714921e-02, 8.68715359e-02, 7.93050594e-02,
       7.23289373e-02, 6.59025762e-02, 5.99877902e-02, 5.45486817e-02,
       4.95515260e-02, 4.49646617e-02, 4.07583846e-02, 3.69048463e-02,
       3.33779574e-02, 3.01532943e-02, 2.72080096e-02, 2.45207477e-02,
       2.20715624e-02, 1.98418391e-02, 1.78142207e-02, 1.59725357e-02,
       1.43017309e-02, 1.27878059e-02, 1.14177520e-02, 1.01794927e-02,
       9.06182795e-03, 8.05438073e-03, 7.14754617e-03, 6.33244339e-03,
       5.60086965e-03, 4.94525689e-03, 4.35863042e-03, 3.83456989e-03,
       3.36717224e-03, 2.95101667e-03, 2.58113150e-03, 2.25296294e-03,
       1.96234547e-03, 1.70547405e-03, 1.47887786e-03, 1.27939557e-03,
       1.10415213e-03, 9.50536945e-04, 8.16183371e-04, 6.98949522e-04,
       5.96900277e-04, 5.08290460e-04, 4.31549118e-04, 3.65264850e-04,
       3.08172140e-04, 2.59138625e-04, 2.17153274e-04, 1.81315403e-04,
       1.50824504e-04, 1.24970827e-04, 1.03126681e-04, 8.47384071e-05,
      

1.0

In [18]:
#gerando as pmfs

#probability mass function
pmf = np.empty((grid,grid))


for t, θt in enumerate(hvals):
    pmf[t,:] = pmf_cdf(θt)
    
#precisamos de uma pmf para quando temos lkt e queremos saber θk,t+1
#chamaremos de pmf2
pmf2 = np.empty((grid, grid))

for i in range(len(hvals)):
    for j in range(len(hvals)):
        pmf2[i, j] = np.sum(pmf[i, :] * pmf[:, j])


Arrays dos valores de continuação dos jogadores

In [19]:
#continuation values
#period, θkt, ℓkt, ℓmt
s_W = np.zeros((T, grid, grid, grid))
j_W = np.copy(s_W)


#optimal payments
#period, θkt, ℓmt, outputs
Pst_array = np.zeros((T, grid, grid, 5))
Pjt_array = np.copy(Pst_array)

In [20]:
#populating the last period with the liquidation values
s_W[(T-1), ...] = s_L(T)
j_W[(T-1), ...] = j_L(T)

Funções para calcular os valores de continuação dos jogadores

In [49]:
#função para mapear os valores das habilidades aos arrays
def find(h):
    
    return nb.int32(np.searchsorted(hvals, h))

find(hvals[7])
type(find(hvals[7]))

find(hvals)

7

numpy.int32

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [50]:
#array to tell us the size of the pie on next period
#period, θk,t+1
U = np.zeros((T, grid))

U = (hvals[:,None] * V[None,:] - C).T


#test
U[1,0] == hvals[0] * V[1] - Ct(1)


U[2,5] == hvals[5] * V[2] - Ct(2)

True

True

In [51]:
#funções genéricas


In [52]:
#função cutoff

def cutoff_m(t, Pkt, hkt, lmt, m_W):
    
    cmt = find(lmt)
    
    while(Pkt > m_W[t, cmt, cmt, find(hkt)] and hvals[cmt] < hhigh):
        cmt += 1
            
    #returns the index of the cutoff
    return max(int32(cmt), find(lmt))


cutoff_m(T-1, 0.05, 0.5, 0.4, j_W)

99

In [61]:
#função payoff ótimo

#retorna a política ótima também

def Pkt(t, hkt, lmt, k_W, m_W, k_L):
    
    #grid of payments is the possible continuation values of the opponent
    Pkt_grid = m_W[t, find(lmt):, find(lmt), find(hkt)]
    
    
    cutoffs = int32(np.empty(Pkt_grid.shape))
    
    #loop for calculating the cutoffs
    for i, Pkt in enumerate(Pkt_grid):
        cutoffs[i] = cutoff_m(t, Pkt, hkt, lmt, m_W)
        
    #cálculo de E1####

    pE1 = np.array(U[t, None, find(hkt):] - Pkt_grid[:, None])
    
    #array com as probabilidades
    
    #matriz com as probabilidades

    #probabiilidades de k amanhã dado a habilidade verdadeira hoje
    probk = pmf[find(hkt),find(hkt):]

    #probabilidade de m amanhã, dado o lower bound hoje
    probm = pmf2[find(lmt), find(lmt):]

    #tipo de produto de kronecker do wikipedia
    #exemplo didático
    # c = np.array([[1], [2]])
    # d = np.array([[1,2], [3,4]])

    # c
    # d

    # np.outer(c,d)

    prob = np.outer(probk,probm)
    
    #multiplicando pE1 pelo array de probabilidades
    pE1 = np.dot(pE1, prob)

    #array indicador
    IE1 = np.where(m_W[t, find(lmt):, cutoffs, find(hkt)].T <= Pkt_grid[:,None] , 1, 0)

    #payoff de E1
    E1 = np.multiply(pE1, IE1)
    
    #soma ao longo das colunas (axis = 1) para que cada linha tenha o payoff esperado de propor um pagamento aceitável
    E1 = np.sum(E1, axis = 1)
    
    
    # #cálculo de E2####

    pE2 = k_W[t, find(hkt):, find(hkt), cutoffs]
    
    #multiplicando pE2 pelo array de probabilidades
    pE2 = np.dot(pE2, prob)
    
    #array indicador de 
    IE2 = np.where(IE1 == 1, 0, 1)
    
    #payoff
    E2 = np.multiply(pE2, IE2)
    E2 = np.sum(E2, axis = 1)

    #matriz dos payoffs####
    matrix_payoff = E1 + E2

    payoff_reorg = np.max(matrix_payoff)
    index = np.argmax(matrix_payoff)

    pagamento = Pkt_grid[index]
    
    
    #calculating the optimal policy between liquidating, reorganizing or waiting ####
    payoff_liq = k_L(t)
    
    payoff_wait = np.sum(np.multiply(pmf[find(hkt), find(hkt):], k_W[t, find(hkt):, find(hkt), find(lmt)]))
    
    
    #0 is liquidating, 1 is reorg, 2 is waiting
    payoff_max = np.max((payoff_liq, payoff_reorg, payoff_wait))
    policy = np.argmax((payoff_liq, payoff_reorg, payoff_wait))
    
                        
                         

    return pagamento, cutoffs[index], payoff_reorg, payoff_max, policy
        

t, hkt, lmt, k_W, m_W, k_L = T-1, 0.5, 0.5, s_W, j_W, s_L

Pkt(t, hkt, lmt, k_W, m_W, k_L)


(0.0, 49, 0.20332358160221173, 0.20338, 0)

In [67]:
def Pst(t, θst, ℓjt):
    return Pkt(t, θst, ℓjt, s_W, j_W, s_L)


def Pjt(t, θjt, ℓst):
    return Pkt(t, θjt, ℓst, j_W, s_W, j_L)

In [68]:
#populando as matrizes para continuar os testes

for h in hvals:
    for l in hvals:
            Pst_array[t, find(h), find(l), :] = Pst(t, h, l)
            Pjt_array[t, find(h), find(l), :] = Pjt(t, h, l)

In [71]:
#checando se populou ok
t, h, l = T-1, 0.5, 0.5
Pst_array[t, find(h), find(l),:] == Pst(t, h, l)

array([ True,  True,  True,  True,  True])

In [72]:
#função da proposta ótima


def propose(t, hkt, lkt, lmt, Pkt_array):
    #apenas busca o valor de Pkt
    
    payoff_max, policy = Pkt_array[t, find(hkt), find(lmt)][-2:]
    
    return payoff_max, policy

#test


t, hkt, lkt, lmt, Pkt_array = T-1, 0.5, 0.01, 0.5, Pst_array

Pst_array[t, find(hkt), find(lmt), :] = Pkt(t, hkt, lmt, k_W, m_W, k_L)

propose(t, hkt, lkt, lmt, Pkt_array)


(0.20338, 0.0)

In [77]:
Pst_array[T-1, :, find(0.5), -1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [89]:
#tentando um negócio com np.where
#procura a primeira entrada do vetor de pagamentos do adversário onde a policy (última entrada) é igual a 1

np.min(np.where(Pst_array[T-1, :, find(0.5), -1] == 1))
#então ele diz que a menor habilidade é a de índice 50. Certeiro



Pst_array[T-1, 49, find(0.5), -1]

Pst_array[T-1, 50, find(0.5), -1]


50

0.0

1.0

In [88]:
a = np.array([1,2,3,4])

np.where(a ==5)

(array([], dtype=int64),)

In [ ]:
Pst_array[T-1, :, find(0.5), -1]

In [101]:
#função de threshold

def threshold_m(t, lkt, lmt, Pmt_array):
    
    threshold_vector = np.where(Pst_array[t, :, find(lkt), -1] == 1)
    
    ts = ts = hvals[np.min(threshold_vector)]
    
    #probability that m will propose liquidation next period
    if(lmt > ts):
        probm_liq = 0
    else:
        probm_liq = cdf(ts,lmt)
        
    #returns the reorganization threshold and the probability that m will propose liquidation tomorrow
    return ts, probm_liq
    
    
 
t, lkt,lmt, Pmt_array = T-1, 0.5,0.51, Pst_array

threshold_m(t, lkt,lmt,  Pmt_array)

(0.51, 0.0)